In [1]:
import json
import jsonlines
import sys
import prompt_rag

dirs = [".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

import covmis

search_engine = "brave"

data_train = covmis.load_train()
data_search = covmis.load_train_search()
data_search_llm = covmis.load_train_llm()


合并多个文件的先验知识

In [2]:
# bing search llm
# v2: k = 5
# v3: 不对时间排序

# brave search llm

# Solar
# v1: k = 5
# v2: k = 5 随机选取

# Mixtral
# v1: k = 5


# llama3
# v1: k = 5, vllm = 0.5.0

sort = False
prior_knowledge_list = []

K = 5
prior_knowledge_version = "1"
model_name = "mixtral"

with open(f"train_search_llm_tmp.json", "r") as f:
    prior_knowledge_list = json.load(f)

for i, item in enumerate(data_search_llm):

    if item["id"] != prior_knowledge_list[i]["id"]:
        print(i)
        print(data_train[i]["claim"])
        print(prior_knowledge_list[i]["claim"])
        raise Exception()
    else:
        # vv = ""
        # if prior_knowledge_list[i].get(f"prior_knowledge_{model_name}3") is not None:
        #     vv = "3"
        # elif prior_knowledge_list[i].get(f"prior_knowledge_{model_name}2") is not None:
        #     vv = "2"

        item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"] = prior_knowledge_list[i][f"prior_knowledge_{model_name}"]
# There was a false study done where they gave it to very sick people -- extremely sick people, people that were ready to die. It was given by, obviously, not friends of the administration. And the study came out. The people were ready to die. Everybody was old, had bad problems with hearts, diabetes, and everything else you can imagine. So they gave it. So, immediately, when it came out, they gave a lot of false information, just so you understand. Great studies came out of Italy on hydroxy. 
# There was a false study done where they gave it to very sick people -- extremely sick people, people that were ready to die. It was given by, obviously, not friends of the administration. And the study came out. The people were ready to die. Everybody was old, had bad problems with hearts, diabetes, and everything else you can imagine. So they gave it. So, immediately, when it came out, they gave a lot of false information, just so you understand. Great studies came out of Italy on hydroxy. 

data_search_llm[2]


{'id': 5,
 'prior_knowledge_llama3_v1_K=5': '**Summary of INFORMATION**\n\nThe provided information consists of five articles and a Wikipedia page that discuss Dr. Judy Mikovits, a former research scientist, and her claims about COVID-19, Dr. Anthony Fauci, and vaccines. The articles fact-check her statements and conclude that they are largely false or misleading.\n\n**Information 1**: A fact-checking article debunks several claims, including the alleged cover-up by Dr. Fauci, the lab-made origin of SARS-CoV-2, and the 36% increase in COVID-19 mortality due to influenza vaccines.\n\n**Information 2**: An article describes Dr. Mikovits\' conspiracy theories as "demonstrably untrue" and mentions that her research was fraudulent.\n\n**Information 3**: A fact-checking article examines Dr. Mikovits\' claims, including her statement that face masks "activate" the virus, and finds no evidence to support her assertions.\n\n**Information 4**: Dr. Mikovits\' Wikipedia page summarizes her backgro

In [3]:
# covmis.save_train_llm(data_search_llm)


创建数据（带有先验知识的Prompt）以微调LLM

In [5]:
# dict_list = []

# data_version = "1"
# for i, item in enumerate(data_search_llm):
    
#     if int(data_train[i]["label"]) != 1:
#         prompt = prompt_rag.get_prompt_with_prior_knowledge(
#             data_train[i]["claim"], 
#             search_engine,
#             data_search[i][f"{search_engine}_search_results"], 
#             item[f"prior_knowledge_{model_name}_v{prior_knowledge_version}_K={K}"], 
#             K=K,
#             claim_date=data_train[i]["date"],
#             known_info=True, 
#             ids=None
#         )
#         label = "TRUE." if int(data_train[i]["label"]) == 2 else "FALSE."
#         dict_list.append({"query": prompt, "response": label})
# print(dict_list[0]["query"])
# len(dict_list), dict_list[:2]


In [6]:
# LLM_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/"
# with jsonlines.open(
#     LLM_dir + \
#     f"swift/examples/pytorch/llm/my_data/with_{model_name}_info/{search_engine}/data{data_version}.jsonl", mode="w") as file_jsonl:
#     for line in dict_list:
#         file_jsonl.write(line)


测试模型的先验知识生成效果：一次提问

In [7]:
import torch
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

# model_type = ModelType.mixtral_moe_7b_instruct

# model_type = CustomModelType.phi_3_medium_4k_instruct
# model_type = CustomModelType.llama_3_70b_instruct_awq
model_type = CustomModelType.mixtral_moe_7b_instruct_awq
# model_type = CustomModelType.solar_instruct_10_7b

llm_engine = get_vllm_engine(
    model_type, 
    # torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    max_model_len=4096,
    # gpu_memory_utilization=0.92,
    # model_id_or_path="/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64, # 64
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=2048,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True
)


2024-07-01 01:26:04,247 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-07-01 01:26:04,249 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-07-01 01:26:04,274 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 21bc0b9ccf26bcf3f4ca2e675ec8875d and a total number of 980 components indexed
[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  

WARNING 07-01 01:26:04 config.py:217] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.


2024-07-01 01:26:08,336	INFO worker.py:1753 -- Started a local Ray instance.


INFO 07-01 01:26:09 config.py:623] Defaulting to use mp for distributed inference
INFO 07-01 01:26:09 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ', speculative_config=None, tokenizer='/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=42, served_model_name=/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ)
INFO 07-01 01:26:09 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 07-01 01:26:09 selector.py:51] Usin

(raylet) [2024-07-01 01:26:17,226 E 124916 124953] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_01-26-04_766329_124564 is over 95% full, available space: 16588029952; capacity: 1967317549056. Object creation will fail if spilling is required.


(VllmWorkerProcess pid=128518) INFO 07-01 01:26:19 model_runner.py:160] Loading model weights took 11.4912 GB
INFO 07-01 01:26:20 model_runner.py:160] Loading model weights took 11.4912 GB


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f42d37d1650>>
Traceback (most recent call last):
  File "/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


INFO 07-01 01:26:23 distributed_gpu_executor.py:56] # GPU blocks: 9029, # CPU blocks: 4096
(VllmWorkerProcess pid=128518) INFO 07-01 01:26:26 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=128518) INFO 07-01 01:26:26 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-01 01:26:26 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-01 01:26:26 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory p

(raylet) [2024-07-01 01:26:27,240 E 124916 124953] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_01-26-04_766329_124564 is over 95% full, available space: 16587997184; capacity: 1967317549056. Object creation will fail if spilling is required.


In [136]:
# get_resp_list = lambda request_list : inference_vllm(
#     llm_engine, template, request_list, 
#     generation_config=generation_config, 
#     use_tqdm=False, 
#     verbose=True, prompt_prefix="", output_prefix=""
# )

s = "Information Summary:\n\nInformation 1, published on 2020-12-01, discusses a false claim about a Covid-19 vaccine having the ability to \"transfer genetic material\" and manipulate human genes. The claim was shared hundreds of times on Facebook, but experts, such as Dr. Kirsty Short, have confirmed that the mRNA vaccine cannot enter the human genome. Information 2, published on 2020-05-19, mentions false claims about a future Covid-19 vaccine genetically modifying humans. Information 3, with no specific publication date, states that Covid-19 vaccines do not change a person's genes and use messenger RNA or modified adenovirus to trigger an immune response. Information 4, published on 2023-11-01, reiterates that Covid-19 vaccines cannot alter a person's genome and debunks false claims about DNA contamination leading to harmful effects. Information 5, with no specific publication date, highlights the importance of monitoring and addressing vaccine misinformation to prevent vaccine hesitancy.\n\nRestated Claim:\nOn 2020-12-01, experts refuted the false claim that a Covid-19 vaccine can manipulate human genes.\n\nGiven the information available, the claim is TRUE. The Covid-19 vaccines developed by Pfizer-BioNTech, Moderna, and Johnson & Johnson do not have the ability to manipulate human genes, as confirmed by various experts and sources. These vaccines either use messenger RNA or modified adenovirus to trigger an immune response, but they cannot alter human DNA."

prompt_list = ["how are you"] * 10

resp_list = get_resp_list([{'query': prompt} for prompt in prompt_list])

for resp in resp_list:
    print(resp["response"])

# print(s)


100%|██████████| 10/10 [00:01<00:00,  6.36it/s]

I'm just a computer program, so I don't have feelings or emotions. I don't have the ability to "be." I exist only to process and provide information. How can I help you today?
I'm just a computer program, so I don't have feelings or emotions. I don't have the ability to "be." I exist only to process and provide information. How can I help you today?
I'm just a computer program, so I don't have feelings or emotions. I don't have the ability to "be." I exist only to process and provide information. How can I help you today?
I'm just a computer program, so I don't have feelings or emotions. I don't have the ability to "be." I exist only to process and provide information. How can I help you today?
I'm just a computer program, so I don't have feelings or emotions. I don't have the ability to "be." I exist only to process and provide information. How can I help you today?
I'm just a computer program, so I don't have feelings or emotions. I don't have the ability to "be." I exist only to pro

In [9]:
# def get_prompt_for_generating_prior_knowledge(
#         claim, claim_date, search_engine, search_results, model_name,
#         K=5, sort=False, ids=None, without_info=False, without_claim_date=False):
#     """
#     sort: 对search result 按时间进行排序
#     """

#     claim = claim.strip()

#     if model_name == "solar":
#         pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.\n\n"
#     elif model_name == "mixtral":
#         # v1
#         # pre = "Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the CLAIM and the date of publication of the INFORMATION.\n\n"
#         # v2
#         # pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\n\n"
        
#         pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, restate the CLAIM, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\n\n"

#     else:
#         raise Exception("model_name 只能从solar，mixtral中选择")
    
#     if without_claim_date:
#         text = "CLAIM: " + claim
#     else:
#         text = "CLAIM: " + prompt_rag.get_claim_with_date(claim, claim_date)

#     if search_engine == "bing":
#         snippet = prompt_rag.get_bing_snippet_v2(search_results, K=K, claim_date=claim_date, sort=sort)
#     elif search_engine == "brave":
#         if ids is None:
#             ids = slice(0, K)
#         snippet = prompt_rag.get_brave_snippet(search_results, ids=ids)
#     else:
#         raise Exception("Select search engines in [\"bing\", \"brave\"].")
    
#     info = "INFORMATION:\n" + snippet + '\n'

#     if without_info:
#         return (pre + text).strip()
#     else:
#         return pre + info + text

# def get_claim_with_date(claim, claim_date=None):
#     if claim_date is None:
#         return " " + claim
    
#     # res = "\n"
#     res = claim + "\nPublication date: " + claim_date
#     return res

# K = 5
# def get_id(claim):
#     for i in range(len(data_train)):
#         if claim.strip() in data_train[i]["claim"].strip():
#             return i

# # i = 0
# i = get_id("False claim circulates that Pakistani plane transported Sri Lankan students home after COVID-19 lockdown")

# # claim = data_search[i]["claim"]
# search_results = data_search[i][f"{search_engine}_search_results"]

# model_name = 'mixtral'
# prompt_list1 = [
#     get_prompt_for_generating_prior_knowledge(
#         data_train[i]["claim"], data_train[i]["date"], search_engine, search_results, model_name,
#         K=K, sort=False, 
#         # ids=data_search[i]["random_ids"],
#         ids=None
#     ),
# ]

# request_list1 = [{'query': prompt} for prompt in prompt_list1]

# print(prompt_list1[0])
# print()

# resp_list1 = get_resp_list(request_list1)
# print(resp_list1[0]["response"].strip())
# print()
# resp_list1[0]["response"].strip()


In [8]:
def get_prompt_for_generating_prior_knowledge2(
        claim, claim_date, search_engine, search_results, model_name,
        K=5, sort=False, ids=None, without_info=False, without_claim_date=False):
    """
    pre + info + text
    """

    claim = claim.strip()

    if model_name == "solar":
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.\n\n"
    elif model_name == "mixtral":
        # pre = "Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a detailed summary of the given INFORMATION and restate the CLAIM. Then reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge. In reasoning, it is necessary to consider the sequential relationship between the date of publication of the CLAIM and the date of publication of the INFORMATION.\n\n"
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\n\n"
        
        # pre = "Below is some INFORMATION searched online and a <CLAIM>. These pieces of INFORMATION are relevant to the <CLAIM>. This <CLAIM> and all INFORMATION include their respective publication dates and contents. To classify the <CLAIM> more accurately (if the content described by the <CLAIM> is correct, it will be classified as TRUE; if the content described by the <CLAIM> is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, and provide reasonable evidence to judge the correctness of the <CLAIM> based on the available information and your knowledge.\n\n"
    
    elif model_name == "llama3":
        # pre = "Below is some INFORMATION searched online and a <CLAIM>. These pieces of INFORMATION are relevant to the <CLAIM>. This <CLAIM> and all INFORMATION include their respective publication dates and contents. To classify the <CLAIM> more accurately (if the content described by the <CLAIM> is correct, it will be classified as TRUE; if the content described by the <CLAIM> is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, restate the <CLAIM>, and provide reasonable evidence to judge the correctness of the <CLAIM> based on the available information and your knowledge.\n\n"
        pre = "Below is some INFORMATION searched online and a <CLAIM>. These pieces of INFORMATION are relevant to the <CLAIM>. This <CLAIM> and all INFORMATION include their respective publication dates and contents. To classify the <CLAIM> more accurately (if the content described by the <CLAIM> is correct, it will be classified as TRUE; if the content described by the <CLAIM> is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, and provide reasonable evidence to judge the correctness of the <CLAIM> based on the available information and your knowledge.\n\n"
    elif model_name == "phi3":
        # CLAIM
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\n\n"
        
        # <CLAIM>
        # pre = "Below is some INFORMATION searched online and a <CLAIM>. These pieces of INFORMATION are relevant to the <CLAIM>. This <CLAIM> and all INFORMATION include their respective publication dates and contents. To classify the <CLAIM> more accurately (if the content described by the <CLAIM> is correct, it will be classified as TRUE; if the content described by the <CLAIM> is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, and provide reasonable evidence to judge the correctness of the <CLAIM> based on the available information and your knowledge.\n\n"

    else:
        raise Exception("model_name 只能从solar，mixtral中选择")
    
    if without_claim_date:
        text = "CLAIM: " + claim
    else:
        if model_name == "mixtral":
            text = "CLAIM: "
        elif model_name == "llama3":
            text = "<CLAIM>: "
        elif model_name == "phi3":
            text = "CLAIM: "
        text += get_claim_with_date(claim, claim_date)

    if search_engine == "bing":
        snippet = prompt_rag.get_bing_snippet_v2(search_results, K=K, claim_date=claim_date, sort=sort)
    elif search_engine == "brave":
        if ids is None:
            ids = slice(0, K)
        snippet = prompt_rag.get_brave_snippet(search_results, ids=ids)
    else:
        raise Exception("Select search engines in [\"bing\", \"brave\"].")
    
    info = "INFORMATION:\n" + snippet + '\n'

    if without_info:
        return (pre + text).strip()
    else:
        return pre + info + text

def get_prompt_for_generating_prior_knowledge1(
        claim, claim_date, search_engine, search_results, model_name,
        K=5, sort=False, ids=None, without_info=False, without_claim_date=False):
    """
    pre + text + info
    """

    claim = claim.strip()

    if model_name == "solar":
        pre = "Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.\n\n"
    elif model_name == "mixtral":


        pre = "Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, restate the CLAIM, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.\n\n"
        pre += "CLAIM: "

        # pre = "Below is a <CLAIM> and some INFORMATION searched online. These pieces of INFORMATION are relevant to the <CLAIM>. This <CLAIM> and all INFORMATION include their respective publication dates and contents. To classify the <CLAIM> more accurately (if the content described by the <CLAIM> is correct, it will be classified as TRUE; if the content described by the <CLAIM> is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, restate the <CLAIM>, and provide reasonable evidence to judge the correctness of the <CLAIM> based on the available information and your knowledge.\n\n"
        # pre += "<CLAIM>: "

    elif model_name == "llama3":

        pre = "Below is a <CLAIM> and some INFORMATION searched online. These pieces of INFORMATION are relevant to the <CLAIM>. This <CLAIM> and all INFORMATION include their respective publication dates and contents. To classify the <CLAIM> more accurately (if the content described by the <CLAIM> is correct, it will be classified as TRUE; if the content described by the <CLAIM> is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, restate the <CLAIM>, and provide reasonable evidence to judge the correctness of the <CLAIM> based on the available information and your knowledge.\n\n"
        pre += "<CLAIM>: "
    
    else:
        raise Exception("model_name 只能从solar，mixtral中选择")
    
    if without_claim_date:
        text = "CLAIM: " + claim
    else:
        text = get_claim_with_date(claim, claim_date) + '\n\n'

    if search_engine == "bing":
        snippet = prompt_rag.get_bing_snippet_v2(search_results, K=K, claim_date=claim_date, sort=sort)
    elif search_engine == "brave":
        if ids is None:
            ids = slice(0, K)
        snippet = prompt_rag.get_brave_snippet(search_results, ids=ids)
    else:
        raise Exception("Select search engines in [\"bing\", \"brave\"].")
    
    info = "INFORMATION:\n" + snippet

    if without_info:
        return (pre + text).strip()
    else:
        return pre + text + info
    
def get_claim_with_date(claim, claim_date=None):
    if claim_date is None:
        return " " + claim
    
    # res = "\n"
    res = claim + "\nPublication date: " + claim_date
    return res

K = 5
def get_id(claim):
    for i in range(len(data_train)):
        if claim.strip() in data_train[i]["claim"].strip():
            return i

# i = 0
i = get_id("Pfizer and Moderna do call their COVID-19 shot a ‘vaccine’")

# ids = [194]
ids1 = [19, 36, 57, 85, 98, 107, 116, 117, 125, 129, 194, 195, 204, 208, 212, 235, 279, 324, 328]
# ids1 = [
#     get_id("Pfizer and Moderna do call their COVID-19 shot a ‘vaccine’"),
#     get_id("Myth spreads online that Australian supermarkets have banned Chinese nationals during COVID-19 pandemic"),
#     get_id("Trump makes false claims about COVID-19 testing"),
#     get_id("UNHCR condemns fake notice which claimed refugees in Malaysia are resisting COVID-19 tests"),
#     get_id("Hospitals in Ohio were not set on fire during protests"),
#     get_id("A 2010 study on vaccines did not show that one in forty were damaged by vaccination"),
#     get_id("The Philippine health department said it did not issue this 'checklist' for COVID-19 symptoms"),
#     get_id("Contaminated CDC COVID-19 test kits recalled and did not spread virus"),
#     get_id("Experts refute false claim that Covid-19 vaccine can 'manipulate' human genes"),
#     get_id("Countries were not buying Covid-19 test kits in 2018"),
#     get_id("False claim circulates that Pakistani plane transported Sri Lankan students home after COVID-19 lockdown"),
#     get_id("People will not have to be vaccinated against COVID-19 to receive food stamps and rent assistance"),
#     get_id("Britain has not awarded a contract to develop a vaccine passport"),
#     get_id("World Health Organization says COVID-19 means ‘coronavirus disease 2019’ – not 'China outbreak virus'"),
#     get_id("The common cold is not the same as COVID-19 and the NHS is not saying it is"),
#     get_id("Philippine authorities did not issue this warning after the novel coronavirus outbreak"),
#     get_id("No tourists have been allowed to visit New Zealand since March 2020 -- this photo has circulated online since 2016"),
#     get_id("This video does not show social distancing failure on an Air India flight during the coronavirus pandemic"),
#     get_id("This photo shows a Sri Lankan airline pilot who tested positive for the novel coronavirus."), # fALSE
# ]

ids2 = [
    get_id("Gynecological Cancers Not Tied to Severe COVID-19"),
    get_id("In February 2021, the Centers for Disease Control and Prevention (CDC) advised U.S. travelers to \u201cavoid all travel to Mexico.\u201d"),
    get_id("The U.S. CDC encourages the use of a \u201c[COVID-19] flu shot\u201d on children."),
    get_id("An announcement was made on March 9 that all classes in the Basque Country were canceled due to the coronavirus."),
    get_id("Rita Wilson, Tom Hanks’ wife, stated in an interview at CBS that “she wouldn’t be alive if not for chloroquine”"),
    get_id("Japan’s Nobel Prize winning Professor of Medicine, Professor Dr Tasuku Honjo has claimed that the coronavirus is not natural and that China manufactured it."),
    get_id("Lies spread by Serbian authorities about COVID-19. For example, the virus does not affect pregnant women, children and young people; no newborn is infected with the coronavirus; the virus does not last long on objects; the number of respiratory machines that Serbia possesses, etc."),
    get_id("Chief Secretary of West Bengal was not following relaxing while the state was performing poorly as it fought COVID-19."),
    get_id("Businessman Ratan Tata has said that 2020 is the year to survive and not care about profits and losses."),
    get_id("Celebrities spreading misinformation about coronavirus and the Janata curfew in India."),
    get_id("This 3 year old girl is fighting for her life after getting the coronavirus."),
    get_id("President Vucic claims that no one said coronavirus is the “funniest” virus."),
    get_id("News photo from stay-at-home protest was doctored to add Confederate flag."),
    get_id("President Donald Trump referred to the coronavirus as a “hoax” or “political conspiracy.”"),
    get_id("President Trump refers to the coronavirus as a hoax in an audio clip."),
    get_id("A false image says that President Duque will declare a shutdown on November 1st."),
    get_id("Professor Perronne makes several false claims on PCR tests, HCQ and hospitals."),
    get_id("Audio of Biden calling the coronavirus a hoax."),
    get_id("President Trump’s claim that he inherited no ventilators from the Obama administration."),
    get_id("Video shows President Donald Trump saying COVID-19 is Democrats’ “new hoax.”"),
    get_id("Fauci Wasn't Involved in New Testing Guidelines"),
    get_id("This movie “Songbird” has been made before Covid. We’ve seen a lot of similar movies, yes. But, it is interesting that in the movie the virus is called COVID-23. Stop the scene when there is news on TV and you will see. Coincidence?"),
    get_id("Images of the newspaper front pages sharing vaccine misinformation"),
    get_id("I don't believe illegal casinos are operating in Bangkok, but if the doctor knows about it, he can inform the authorities."),
    get_id("Facebook posts promote false conspiracy that coronavirus testing patent was submitted in 2015"),
    get_id("Misleading description of Canada\u2019s quarantine sites feeds Covid-19 conspiracy"),
    get_id("Video repeats COVID-19 conspiracy theories"),
    get_id("The post contains a COVID-19 conspiracy theory written by former South Carolina Rep. Trey Gowdy."),
    get_id("A long conspiracy video claims that: vaccines can alter a person’s DNA, nanorobots are inserted with the vaccine to collect biometric data, Bill Gates already owns this data and the body can receive 5G signal after the vaccine is taken"),
    get_id("Jorge Luis Sonnante published a 16-minute video that went viral on networks. In the video, Sonnante (who describes himself as a \u201cdeacon\u201d, but gives falsified evidence of this charge) mixes several conspiracy theories, some meaningless, others already denied, about the pandemic caused by the SARS coronavirus -CoV-2."),
    get_id("Trump touts testing as \"greatest capacity in the world,\" but says people \"shouldn't want to get tested\""),
    get_id("Alberto Fernández, about the coronavirus: “Mortality in people over 65 is 80%”."),
    get_id("An English man received a COVID-19 vaccine through his shirt"),
    get_id("The first recipient of the COVID-19 trial vaccine is dead."),
    get_id("ICUs are not overwhelmed"),
    get_id("Coronavirus tests do not work to diagnose COVID-19. Red Bull tests positive."),
    get_id("Rapidly developed pharmaceuticals such as the Covid-19 vaccines compared to a sedative from the 1950s that caused serious birth defects."),
    get_id("The major cause of death in Covid-19 is thrombosis or blood clot and not pneumonia."),
    get_id("The COVID-19 virus was artificially created by the U.S. military and placed in a capsule."),
    get_id("A video of a girl collapsing in a store is being shared widely on social media with a claim that the girl died of Coronavirus."),
    get_id("Italy\u2019s prime minister cries and declares that his country \u201clost the battle\u201d against the coronavirus."),
    get_id("Black people are not being targeted for UK coronavirus vaccine trials"),
    get_id("In September 2019, a train named COVID-19 appeared in the United States, carrying gases that cause COVID-19."),
    get_id("If you take the vaccine, you'll be enrolled in a pharmacovigilance tracking system. It means that you've enrolled yourself in a medical trial."),
    get_id("Henry Kissinger quote about mandatory vaccinations"),
    get_id("RECEIVE FOOD AID SOCIAL PLAN 2020"),
    get_id("Post claims the video clip is last message of deceased Pakistani doctor Osama Riyaz who had contracted Coronavirus while treating patients"),
    get_id("For a few days now, an image has been circulating on Facebook claiming that COVID-19 can be cured with Dolex Gripa, \"Noraver\" night, and gargling with warm water and lemon."),
    get_id("A magic remedy to prevent COVID-19: mix brown sugar, ginger, garlic, and Chinese leek, boil them and drink it."),
    get_id("Some statements by the American political scientist and activist Susan George in which she says that “Spaniards are laboratory rats: let’s see how much punishment they tolerate without rebelling.” The phrase is shared related to the current coronavirus pandemic."),
    get_id("Saddam Hussein explains what the coronavirus is."),
    get_id("A Facebook post is spreading the false claim that former President Barack Obama gave $3.8 million to a lab in Wuhan, China."),
    get_id("\u2019No discrimination\u2019 against Africans amid pandemic.")
]

ids_x = [ids2[3],]

prompt_list = []
model_name = 'mixtral'

# [ids2[ii] for ii in ids_filtered]:
for i in ids2:
    # claim = data_search[i]["claim"]
    search_results = data_search[i][f"{search_engine}_search_results"]
    prompt_list.append(get_prompt_for_generating_prior_knowledge2(
            data_train[i]["claim"], data_train[i]["date"], search_engine, 
            search_results, model_name, K=K, sort=False, 
            # ids=data_search[i]["random_ids"],
            ids=None
        ))
request_list = [{'query': prompt} for prompt in prompt_list]

resp_list = get_resp_list(request_list)
# print(resp_list[0]["response"].strip())


100%|██████████| 53/53 [07:10<00:00,  8.12s/it]


In [40]:
# llama-3-70b
# CLAIM + info wrong: 1, 18
# info + CLAIM wrong: 18

# CLAIM + info wrong: 1, 2, 3, 7, 11, 15, 20, 30, 45, 51
# info + CLAIM wrong: 1, 2, $3, 7, 11, 20, 45  √√√√√√√√√√√


# mixtral-8x7b
# claim wrong: 0, 1, 8, 10, 12
# <claim>:1, 3

# claim: 0, 1, 15, 16, 20, 24, 25, 30, 51
# <claim>: 0, 1, 2, 7, 15, 16, 24, 25, 29, 51

# phi-3-medium
# claim wrong: 
# <claim>: 1, 8, 13, 18

# claim: 1, 3, 7, 13, 15, 20, 22, 24, 25, 39, 51
# <claim>: 


print(len(ids2))

nn = 29
print(prompt_list[nn])
# print('*'*50)
# print(data_train[ids2[nn]]["label"])
# print(resp_list[nn]["response"].strip())



53
Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.

INFORMATION:
Information 1:
Publication date: 2020-05-14
Title: FALSE: Jorge Luis Sonnante published a 16-minute video that went ...
Content:
The #CoronavirusFacts database records fact-checks published since the beginning of the COVID-19 outbreak. The pandemic and its consequences are constantly evolving and data that was accurate weeks or even days ago might have changed. Remember to check the date w